# xxx

In [172]:
%load_ext tensorboard

In [3]:
from tqdm import tqdm
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import time
def set_style():
    sns.set_style('white')
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['figure.figsize'] = [10, 10]

    pd.options.display.max_columns = 300
    pd.options.display.max_rows = 1000
    # plt.rcParams['figure.dpi'] = 200
    sns.set_palette('muted')  # 调色板颜色温和
    sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1})

set_style()
import sys

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import keras
from keras import Sequential
from keras import layers
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers import Conv1D,MaxPooling1D, Flatten, BatchNormalization
from keras.layers import concatenate

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [6]:
!ls data

track1_round1_testA_20210222.csv  track1_round1_train_20210222.csv


In [7]:
test_data = open('data/track1_round1_testA_20210222.csv').readlines()
test_data = [data.strip().split('|,|') for data in test_data]
test_dataset = pd.DataFrame(test_data, columns=['report_id', 'description'])
test_dataset['words'] = test_dataset.description.apply(lambda x:x.split())

test_dataset.head()

,report_id,description,words
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[852, 328, 697, 538, 142, 355, 582, 800, 728, ..."
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[380, 358, 343, 654, 171, 832, 47, 832, 690, 6..."
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,"[751, 335, 834, 582, 717, 583, 585, 693, 623, ..."
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,"[623, 328, 649, 582, 488, 12, 578, 623, 538, 3..."
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,"[83, 293, 398, 797, 382, 363, 145, 424, 693, 6..."


In [8]:
train_data = open('data/track1_round1_train_20210222.csv','r').readlines()

train_data = [data.strip().split('|,|')  for data in train_data]

train_dataset = pd.DataFrame(train_data,columns=['report_id','description','label'])
train_dataset['words'] = train_dataset.description.apply(lambda x:x.split())

train_dataset.head()

,report_id,description,label,words
0,0,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[623, 328, 538, 382, 399, 400, 478, 842, 698, ..."
1,1,48 328 538 382 809 623 434 355 382 382 363 145...,,"[48, 328, 538, 382, 809, 623, 434, 355, 382, 3..."
2,2,623 656 293 851 636 842 698 493 338 266 369 69...,15,"[623, 656, 293, 851, 636, 842, 698, 493, 338, ..."
3,3,48 328 380 259 439 107 380 265 172 470 290 693...,,"[48, 328, 380, 259, 439, 107, 380, 265, 172, 4..."
4,4,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[623, 328, 399, 698, 493, 338, 266, 14, 177, 4..."


In [9]:
dataset = train_dataset.append(test_dataset)
dataset.head()

,report_id,description,label,words
0,0,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[623, 328, 538, 382, 399, 400, 478, 842, 698, ..."
1,1,48 328 538 382 809 623 434 355 382 382 363 145...,,"[48, 328, 538, 382, 809, 623, 434, 355, 382, 3..."
2,2,623 656 293 851 636 842 698 493 338 266 369 69...,15,"[623, 656, 293, 851, 636, 842, 698, 493, 338, ..."
3,3,48 328 380 259 439 107 380 265 172 470 290 693...,,"[48, 328, 380, 259, 439, 107, 380, 265, 172, 4..."
4,4,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[623, 328, 399, 698, 493, 338, 266, 14, 177, 4..."


In [14]:
mlb = MultiLabelBinarizer()

df_label = pd.DataFrame(mlb.fit_transform(train_dataset.label.apply(lambda x:x.split())))
df_label.columns = mlb.classes_


label = df_label[[str(i) for i in range(17)]].values
label.shape

(10000, 17)

In [16]:
df_label[[str(i) for i in range(17)]].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [17]:
tokenizer=Tokenizer()  #创建一个Tokenizer对象
#fit_on_texts函数可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小
tokenizer.fit_on_texts(dataset['words'])
vocab=tokenizer.word_index #得到每个词的编号
# 将每个样本中的每个词转换为数字列表，使用每个词的编号进行编号

In [103]:
maxlen=100

train_data =tokenizer.texts_to_sequences(train_dataset['words'])
#将超过固定值的部分截掉，不足的在最前面用0填充
train_data_padded_seqs = pad_sequences(train_data, maxlen=maxlen, value = len(vocab))

test_data =tokenizer.texts_to_sequences(test_dataset['words'])
#将超过固定值的部分截掉，不足的在最前面用0填充
test_data_padded_seqs = pad_sequences(test_data, maxlen=maxlen, value = len(vocab))

In [104]:
train_data_padded_seqs.shape

(10000, 100)

## CNN

In [71]:
import keras

def my_score(y_true,y_pred):
    return 1 - keras.losses.binary_crossentropy(y_true, y_pred)

In [72]:
model = Sequential()
model.add(Embedding(len(vocab) + 1, 300, input_length=100)) #使用Embeeding层将每个词编码转换为词向量
model.add(Conv1D(256, 5, padding='same'))
model.add(MaxPooling1D(3, 3, padding='same'))
model.add(Conv1D(128, 5, padding='same'))
model.add(MaxPooling1D(3, 3, padding='same'))
model.add(Conv1D(64, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(BatchNormalization())  # (批)规范化层
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(17, activation='softmax'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=5, batch_size=128, validation_split=0.1)
# y_predict = model.predict_classes(x_test_padded_seqs)  # 预测的是类别，结果就是类别号
# y_predict = list(map(str, y_predict))
# print('准确率', metrics.accuracy_score(y_test, y_predict))
# print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 100, 300)          257700    
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 100, 256)          384256    
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 34, 256)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 34, 128)           163968    
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 12, 128)           0         
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 12, 64)            24640     
_________________________________________________________________
flatten_14 (Flatten)         (None, 768)             

## textCnn

In [169]:
import datetime

In [179]:
def textcnn(maxlen):
    main_input = Input(shape=(maxlen,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(len(vocab) + 1, 512, input_length=maxlen, trainable=False)
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn_ = Conv1D(256, 1, padding='same', strides=1, activation='relu')(embed)
    cnn_ = MaxPooling1D(pool_size=maxlen)(cnn_)

    cnn0 = Conv1D(256, 2, padding='same', strides=1, activation='relu')(embed)
    cnn0 = MaxPooling1D(pool_size=maxlen-1)(cnn0)
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=maxlen-2)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=maxlen-3)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=maxlen-4)(cnn3)

    cnn4 = Conv1D(256, 6, padding='same', strides=1, activation='relu')(embed)
    cnn4 = MaxPooling1D(pool_size=maxlen-5)(cnn4)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn_, cnn0, cnn1, cnn2, cnn3, cnn4], axis=-1)
    
    flat = Flatten()(cnn)
    drop1 = Dropout(0.5)(flat)
#     norm = BatchNormalization()(drop1)
#     dense = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(norm)
# #     drop2 = Dropout(0.5)(dense)
    main_output = Dense(17, activation='softmax')(drop1)
    model = Model(inputs=main_input, outputs=main_output)
    
    return model

In [180]:
model = textcnn(maxlen)
model.summary()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystop_callback = EarlyStopping(monitor='val_loss', patience=20)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=300, batch_size=800,validation_split=0.1,
          callbacks=[tensorboard_callback, earlystop_callback])

Model: "model_57"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_61 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_82 (Embedding)        (None, 100, 512)     439808      input_61[0][0]                   
__________________________________________________________________________________________________
conv1d_314 (Conv1D)             (None, 100, 256)     131328      embedding_82[0][0]               
__________________________________________________________________________________________________
conv1d_315 (Conv1D)             (None, 100, 256)     262400      embedding_82[0][0]               
___________________________________________________________________________________________

12/12 [==============================] - 2s 182ms/step - loss: 0.0558 - my_score: 0.9442 - val_loss: 0.0563 - val_my_score: 0.9437
Epoch 27/300
12/12 [==============================] - 2s 183ms/step - loss: 0.0536 - my_score: 0.9464 - val_loss: 0.0546 - val_my_score: 0.9454
Epoch 28/300
12/12 [==============================] - 2s 183ms/step - loss: 0.0512 - my_score: 0.9488 - val_loss: 0.0535 - val_my_score: 0.9465
Epoch 29/300
12/12 [==============================] - 2s 186ms/step - loss: 0.0487 - my_score: 0.9513 - val_loss: 0.0519 - val_my_score: 0.9481
Epoch 30/300
12/12 [==============================] - 2s 183ms/step - loss: 0.0477 - my_score: 0.9523 - val_loss: 0.0509 - val_my_score: 0.9491
Epoch 31/300
12/12 [==============================] - 2s 182ms/step - loss: 0.0466 - my_score: 0.9534 - val_loss: 0.0495 - val_my_score: 0.9505
Epoch 32/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0444 - my_score: 0.9556 - val_loss: 0.0483 - val_my_score: 0.9517
Epoch

Epoch 83/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0099 - my_score: 0.9901 - val_loss: 0.0295 - val_my_score: 0.9705
Epoch 84/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0099 - my_score: 0.9901 - val_loss: 0.0293 - val_my_score: 0.9707
Epoch 85/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0102 - my_score: 0.9898 - val_loss: 0.0287 - val_my_score: 0.9713
Epoch 86/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0096 - my_score: 0.9904 - val_loss: 0.0299 - val_my_score: 0.9701
Epoch 87/300
12/12 [==============================] - 2s 186ms/step - loss: 0.0096 - my_score: 0.9904 - val_loss: 0.0287 - val_my_score: 0.9713
Epoch 88/300
12/12 [==============================] - 2s 187ms/step - loss: 0.0091 - my_score: 0.9909 - val_loss: 0.0280 - val_my_score: 0.9720
Epoch 89/300
12/12 [==============================] - 2s 186ms/step - loss: 0.0089 - my_score: 0.9911 - val_loss: 0.0301 - val_my_score:

12/12 [==============================] - 2s 185ms/step - loss: 0.0038 - my_score: 0.9962 - val_loss: 0.0258 - val_my_score: 0.9742
Epoch 140/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0037 - my_score: 0.9963 - val_loss: 0.0276 - val_my_score: 0.9724
Epoch 141/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0039 - my_score: 0.9961 - val_loss: 0.0275 - val_my_score: 0.9725
Epoch 142/300
12/12 [==============================] - 2s 184ms/step - loss: 0.0034 - my_score: 0.9966 - val_loss: 0.0266 - val_my_score: 0.9734
Epoch 143/300
12/12 [==============================] - 2s 185ms/step - loss: 0.0037 - my_score: 0.9963 - val_loss: 0.0268 - val_my_score: 0.9732
Epoch 144/300
12/12 [==============================] - 2s 183ms/step - loss: 0.0037 - my_score: 0.9963 - val_loss: 0.0264 - val_my_score: 0.9736
Epoch 145/300
12/12 [==============================] - 2s 185ms/step - loss: 0.0036 - my_score: 0.9964 - val_loss: 0.0269 - val_my_score: 0.9731

In [182]:
model = textcnn(maxlen)
earlystop_callback = EarlyStopping(monitor='loss', patience=20)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=200, batch_size=800, callbacks=[earlystop_callback])

Epoch 1/200
13/13 [==============================] - 3s 172ms/step - loss: 0.4926 - my_score: 0.5074
Epoch 2/200
13/13 [==============================] - 2s 171ms/step - loss: 0.2808 - my_score: 0.7192
Epoch 3/200
13/13 [==============================] - 2s 172ms/step - loss: 0.2353 - my_score: 0.7647
Epoch 4/200
13/13 [==============================] - 2s 174ms/step - loss: 0.1947 - my_score: 0.8053
Epoch 5/200
13/13 [==============================] - 2s 172ms/step - loss: 0.1665 - my_score: 0.8335
Epoch 6/200
13/13 [==============================] - 2s 174ms/step - loss: 0.1475 - my_score: 0.8525
Epoch 7/200
13/13 [==============================] - 2s 171ms/step - loss: 0.1318 - my_score: 0.8682
Epoch 8/200
13/13 [==============================] - 2s 172ms/step - loss: 0.1205 - my_score: 0.8795
Epoch 9/200
13/13 [==============================] - 2s 175ms/step - loss: 0.1131 - my_score: 0.8869
Epoch 10/200
13/13 [==============================] - 2s 173ms/step - loss: 0.1062 - my_sco

13/13 [==============================] - 2s 174ms/step - loss: 0.0093 - my_score: 0.9907
Epoch 82/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0088 - my_score: 0.9912
Epoch 83/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0091 - my_score: 0.9909
Epoch 84/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0084 - my_score: 0.9916
Epoch 85/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0083 - my_score: 0.9917
Epoch 86/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0081 - my_score: 0.9919
Epoch 87/200
13/13 [==============================] - 2s 175ms/step - loss: 0.0075 - my_score: 0.9925
Epoch 88/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0078 - my_score: 0.9922
Epoch 89/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0074 - my_score: 0.9926
Epoch 90/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0073 - my_score: 

13/13 [==============================] - 2s 174ms/step - loss: 0.0027 - my_score: 0.9973
Epoch 161/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0024 - my_score: 0.9976
Epoch 162/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0024 - my_score: 0.9976
Epoch 163/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0025 - my_score: 0.9975
Epoch 164/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0022 - my_score: 0.9978
Epoch 165/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0024 - my_score: 0.9976
Epoch 166/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0024 - my_score: 0.9976
Epoch 167/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0023 - my_score: 0.9977
Epoch 168/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0022 - my_score: 0.9978
Epoch 169/200
13/13 [==============================] - 2s 174ms/step - loss: 0.0022 - m

In [174]:
%tensorboard --logdir logs/fit --host 0.0.0.0

### kfold

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cvscores = []
models = []

for train, test in kfold.split(train_data_padded_seqs[:,0], label[:,0]):
    model = textcnn(maxlen)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
#     model.fit(train_data_padded_seqs, label, epochs=20, batch_size=800,validation_split=0.1)
    model.fit(train_data_padded_seqs[train], label[train], epochs=50, batch_size=800, verbose=0)
    scores = model.evaluate(train_data_padded_seqs[test], label[test], verbose=0)
    models.append(model)
    cvscores.append(scores)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [128]:
print(cvscores)

[[0.032859619706869125, 0.9671404361724854], [0.03199091553688049, 0.9680091738700867], [0.03177975118160248, 0.9682203531265259], [0.035756807774305344, 0.9642431139945984], [0.03890657797455788, 0.9610933661460876], [0.03947802633047104, 0.9605220556259155], [0.03569938987493515, 0.9643005728721619], [0.03354867920279503, 0.9664511680603027], [0.03180423378944397, 0.9681957364082336], [0.032635584473609924, 0.9673644304275513]]


## Bilstm

In [76]:
inputs = Input(shape=(100,), dtype='float64')
embedder = Embedding(len(vocab) + 1, 300, input_length=100, trainable=False)
embed = embedder(inputs)
# Embed each integer in a 128-dimensional vector
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embed)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = Dense(17, activation='softmax')(x)
model = Model(inputs, outputs)
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=10, batch_size=200,validation_split=0.1)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 100, 300)          257700    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_28 (Dense)             (None, 17)                2193      
Total params: 545,589
Trainable params: 287,889
Non-trainable params: 257,700
_________________________________________________________________
Epoch 1/10
45/45 [==============================] - 5s 60ms/step - loss: 0.4762 - my_score: 0.5238 - val_loss: 0.

## Transformer

In [87]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 256  # Hidden layer size in feed forward network inside transformer
maxlen=100
vocab_size = len(vocab) + 1

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(17, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
# model.fit(train_data_padded_seqs, label, epochs=20, batch_size=200,validation_split=0.1)
model.fit(train_data_padded_seqs, label, epochs=20, batch_size=200)

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
token_and_position_embedding (None, 100, 256)          245504    
_________________________________________________________________
transformer_block_9 (Transfo (None, 100, 256)          658688    
_________________________________________________________________
global_average_pooling1d_9 ( (None, 256)               0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_68 (Dense)             (None, 100)               25700     
_________________________________________________________________
dropout_68 (Dropout)         (None, 100)               0  

## Prediction

In [184]:
test_pred = model.predict(test_data_padded_seqs)

In [185]:
result = ['|%s'%' '.join(['%.8f'%i for i in pred]) for pred in test_pred]

In [186]:
result[0]

'|0.00000003 0.00000013 0.00000001 0.00000000 0.00000000 0.00000000 0.00000000 0.98520273 0.00000898 0.01478814 0.00000002 0.00000000 0.00000000 0.00000000 0.00000000 0.00000001 0.00000000'

In [187]:
test_dataset['report_ID'] = test_dataset['report_id'].apply(lambda x:'%s|'%x)
test_dataset['Prediction'] =result
test_dataset.head()

,report_id,description,words,report_ID,Prediction
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[852, 328, 697, 538, 142, 355, 582, 800, 728, ...",0|,|0.00000003 0.00000013 0.00000001 0.00000000 0...
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[380, 358, 343, 654, 171, 832, 47, 832, 690, 6...",1|,|0.00225043 0.02840945 0.03147091 0.04229768 0...
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,"[751, 335, 834, 582, 717, 583, 585, 693, 623, ...",2|,|0.00000049 0.00000160 0.00000013 0.00000272 0...
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,"[623, 328, 649, 582, 488, 12, 578, 623, 538, 3...",3|,|0.01349525 0.00311254 0.09612284 0.00368371 0...
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,"[83, 293, 398, 797, 382, 363, 145, 424, 693, 6...",4|,|0.00000003 0.00000335 0.00000002 0.00000441 0...


In [188]:
test_dataset[['report_ID','Prediction']].to_csv('output/submit_%d.csv'%(int(time.time())),
                                                index = False,header=None)

## Prediction2

In [131]:
test_pred = []
for model in models:
    test_pred.append(model.predict(test_data_padded_seqs))
test_pred = np.array(test_pred).mean(axis = 0)

In [132]:
result = ['|%s'%' '.join(['%.8f'%i for i in pred]) for pred in test_pred]

In [133]:
result[0]

'|0.00000001 0.00000001 0.00000001 0.00000000 0.00000000 0.00000000 0.00000000 0.99086821 0.00000419 0.00912758 0.00000004 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000'

In [134]:
test_dataset['report_ID'] = test_dataset['report_id'].apply(lambda x:'%s|'%x)
test_dataset['Prediction'] =result
test_dataset.head()

,report_id,description,words,report_ID,Prediction
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[852, 328, 697, 538, 142, 355, 582, 800, 728, ...",0|,|0.00000001 0.00000001 0.00000001 0.00000000 0...
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[380, 358, 343, 654, 171, 832, 47, 832, 690, 6...",1|,|0.00194085 0.00767052 0.00348932 0.03106930 0...
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,"[751, 335, 834, 582, 717, 583, 585, 693, 623, ...",2|,|0.00000038 0.00000087 0.00000012 0.00000024 0...
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,"[623, 328, 649, 582, 488, 12, 578, 623, 538, 3...",3|,|0.00518915 0.00369618 0.07828070 0.00305524 0...
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,"[83, 293, 398, 797, 382, 363, 145, 424, 693, 6...",4|,|0.00000000 0.00000615 0.00000000 0.00002002 0...


In [140]:
test_dataset[['report_ID','Prediction']].to_csv('output/submit_%d.csv'%(int(time.time())),
                                                index = False,header=None)